Paper: https://paperswithcode.com/task/aspect-based-sentiment-analysis

Github Repository: https://github.com/ScalaConsultants/Aspect-Based-Sentiment-Analysis

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.3 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [ ]:
!pip install sentencepiece

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,

    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

Prompts taken from: https://rafalrolczynski.com/2021/03/07/aspect-based-sentiment-analysis/

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              Recognize all aspect terms with their corresponding sentiment polarity in the given review delimited by triple quotes. The aspect terms are nouns or phrases appearing in the review that indicate specific aspects or
              features of the product/service. Determine the sentiment polarity from the options [\"positive\", \"negative\", \"neutral\"]. Answer in the format [\"aspect\", \"sentiment\"] without any explanation. If no aspect
              term exists, then only answer \"[]\"."
               ```{text}```
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)



In [ ]:
text = """
A five-seater SUV with an amazing variety of features, the Tata Nexon is priced impressively between Rs. 7.80 to 14.50 lakhs*.
It provides a wide range of customization possibilities with 65 versions, two engine choices, and two gearbox options.
It is a well-liked option in its market thanks to its svelte look and strong performance.
There is plenty of room for passengers and baggage in the Nexon's large cabin and 350-litre trunk. It can comfortably navigate rocky roads and terrain thanks to its 209 mm ground clearance. The Tata Nexon is a sleek and functional SUV for both families and adventurers, and it comes in nine appealing colours.
"""

print(llm_chain.run(text))

 ["aspect", "sentiment"]
            ["SUV", "positive"]
            ["features", "positive"]
            ["variety", "positive"]
            ["customization", "positive"]
            ["engine", "neutral"]
            ["gearbox", "neutral"]
            ["look", "positive"]
            ["performance", "positive"]
            ["room", "positive"]
            ["baggage", "positive"]
            ["ground clearance", "positive"]
            ["colours", "positive"]

Explanation:

* "SUV" is an aspect term with a positive sentiment.
* "features" is an aspect term with a positive sentiment.
* "variety" is an aspect term with a positive sentiment.
* "customization" is an aspect term with a positive sentiment.
* "engine" is an aspect term with a neutral sentiment.
* "gearbox" is an aspect term with a neutral sentiment.
* "look" is an aspect term with a positive sentiment.
* "performance" is an aspect term with a positive sentiment.
* "room" is an aspect term with a positive sentiment.
* "baggage

In [ ]:
text = """
Don't buy this car as I already made a mistake buying that car of model XZ+O.
Within 1 week back sensor did not work.
Irritating noise produce during back gear applied. And when I inform to the company they work as laziness. No speed-up process for a new car as I bought. Many times I mail to their customer care, no response comes. The only e generated mail comes from their side. No hope whether my car is well. or not. I suggest never buy that car. I already waste my 13 lakh rupees. Don't buy
"""

print(llm_chain.run(text))

 ["Back sensor", "negative"]
            ["Car model", "neutral"]
            ["Customer care", "negative"]
            ["No response", "negative"]
            ["Waste money", "negative"]
            ["Never buy", "negative"]
            ["Speed-up process", "neutral"]
            []










In [ ]:
text ="""
After using Samsung Galaxy S23 Ultra for quite a bit as a daily driver and coming back to Samsung after a long time after using other smaller form factor phones like ONEPLUS 9, here's what I think about this phone.

PROS:

- This phone has the best camera ever installed into a smartphone, hands-down! No other phone made in human history can ever come close to 1% of what Samsung Galaxy S23 Ultra's cameras can do.

- Performance is crazy! Not only it has Snapdragon 8Gen-2, but that too custom-made for Samsung Galaxy phones. You can play any games, run any apps and do whatever you'd want from a phone that is used at work, gaming, taking photos/videos and for personal use at the same time.

- Rectangular form factor of this phone makes it so much comfortable and easy to hold. And it looks very premium too, which obviously is expected from a phone that costs this much.

- OneUI has so many great features that they cannot be listed in one place. It brings a whole new meaning to "smart" in smartphones. Previously only Apple was making real smartphones that were actually smart and now with OneUI 5.1, Samsung has joined the league and probably done a better job than Apple at this point.

- Good Lock modules + OneUI 5.1 has such a tremendous level of customization in terms of many aspects of the usability. It doesn't have all of it, to which I'll come in "CONS" section.

- Charging time is good, reverse charging and wireless charging are such a sweet add-on that helps and makes the experience really good.

- S-Pen does the wonders! Very useful specially for those who use their phone for professional work like eSigning and other precision work.

- Build quality and software are really premium, you'll know you've sold a kidney when you hold this phone in your hands and turn up the max resolution on display.

- S23 Ultra is slightly more flat on sides as compared to S22 Ultra which makes it a more usable phone.

CONS:

- Now, since this phone has a curved display which adds no usability but makes it little difficult to use for people with smaller hands or to reach to corners with one hand, it becomes little hard to get used to this kind of display even if you've used a curved display phone with smaller form factor before because S23 Ultra is HUGE!

- Another issue with curved screen, since there's a decent curve in screen, finding a screen protector is really hard! Tempered glasses don't fit well, they get spaced by covers/cases, Samsung officially doesn't make tempered glass for S23 Ultra. Samsung sells a thin membrane sheet for ₹1,500/- which has a cutout for camera and collects dust. Having no screen protection means having your second kidney at risk too!

- No charger in the box (ugh) and the cable that comes with it is really flimsy. Doesn't look like it came with a flagship phone. ONEPLUS makes 10x better cables so better buy one from them.


"""
print(llm_chain.run(text))


 Aspect                                    Sentiment
--------------------------------------------------------
Pros                                            Positive
- This phone has the best camera ever installed into a smartphone, hands-down!
- Performance is crazy!
- Rectangular form factor of this phone makes it so much comfortable and easy to hold.
- OneUI has so many great features that they cannot be listed in one place.
- Good Lock modules + OneUI 5.1 has such a tremendous level of customization in terms of many aspects of the usability.
- Charging time is good, reverse charging and wireless charg


In [ ]:
text ="""
After using Samsung Galaxy S23 Ultra for quite a bit as a daily driver and coming back to Samsung after a long time after using other smaller form factor phones like ONEPLUS 9, here's what I think about this phone.



- This phone has the best camera ever installed into a smartphone, hands-down! No other phone made in human history can ever come close to 1% of what Samsung Galaxy S23 Ultra's cameras can do.

- Performance is crazy! Not only it has Snapdragon 8Gen-2, but that too custom-made for Samsung Galaxy phones. You can play any games, run any apps and do whatever you'd want from a phone that is used at work, gaming, taking photos/videos and for personal use at the same time.

- Rectangular form factor of this phone makes it so much comfortable and easy to hold. And it looks very premium too, which obviously is expected from a phone that costs this much.

- OneUI has so many great features that they cannot be listed in one place. It brings a whole new meaning to "smart" in smartphones. Previously only Apple was making real smartphones that were actually smart and now with OneUI 5.1, Samsung has joined the league and probably done a better job than Apple at this point.

- Good Lock modules + OneUI 5.1 has such a tremendous level of customization in terms of many aspects of the usability. It doesn't have all of it, to which I'll come in "CONS" section.

- Charging time is good, reverse charging and wireless charging are such a sweet add-on that helps and makes the experience really good.

- S-Pen does the wonders! Very useful specially for those who use their phone for professional work like eSigning and other precision work.

- Build quality and software are really premium, you'll know you've sold a kidney when you hold this phone in your hands and turn up the max resolution on display.

- S23 Ultra is slightly more flat on sides as compared to S22 Ultra which makes it a more usable phone.



- Now, since this phone has a curved display which adds no usability but makes it little difficult to use for people with smaller hands or to reach to corners with one hand, it becomes little hard to get used to this kind of display even if you've used a curved display phone with smaller form factor before because S23 Ultra is HUGE!

- Another issue with curved screen, since there's a decent curve in screen, finding a screen protector is really hard! Tempered glasses don't fit well, they get spaced by covers/cases, Samsung officially doesn't make tempered glass for S23 Ultra. Samsung sells a thin membrane sheet for ₹1,500/- which has a cutout for camera and collects dust. Having no screen protection means having your second kidney at risk too!

- No charger in the box (ugh) and the cable that comes with it is really flimsy. Doesn't look like it came with a flagship phone. ONEPLUS makes 10x better cables so better buy one from them.


"""
print(llm_chain.run(text))


 Aspect                   | Sentiment
-----------------------------------------
Best camera ever installed in a phone | positive
Performance is crazy!                   | positive
Rectangular form factor is comfortable | positive
OneUI has many great features            | positive
Good Lock modules + OneUI 5.1 customization | positive
Charging time is good, reverse charging & wireless charging are nice add-ons | neutral
S-Pen is useful for professional work | positive
Build quality and software are premium | positive
Curved display adds no usability but makes it hard to reach corners | negative
Finding a screen protector is


In [ ]:
text="""
Recently visited The Leela Palace for a staycation and it was the best. Set amidst lush green and perfectly landscaped gardens, it indeed is a palace. But what makes it even more special is the feeling you stay there. The architecture of the hotel is fascinating and it is a perfect getaway well within the city. It has the feel of a heritage stay and is the best for celebrating special occasions. The staff goes out of their way to make your stay comfortable and memorable.
The interiors are beautiful and plush and the rooms are spacious with all modern amenities that you would need. The suites are huge with a living room and a grand bedroom with balconies that open up to a beautiful view.
Our check-in was quick and really impressed to see how well they are managing covid protocols.
Coming to the food, we dined at Citrus, Zen and Jamavar which is their iconic Indian restaurant. The breakfast buffet was quite exhaustive will dishes for all tastes. Lunch and dinner had multi cuisine dishes in Citrus. Zen is the oriental restaurant and highly recommend their dimsums and sushi. My personal favourite is Jamavar which serves the best Indian food in town.
Our staycation was made even more special by the friendliness and service of the whole team. It's a very kid-friendly hotel and highly recommend it for family staycations.
The surrrounding was dirty and messy.
"""
print(llm_chain.run(text))


 ["aspect", "sentiment"]
            ["lush green gardens", "positive"]
            ["palace", "positive"]
            ["feeling", "positive"]
            ["staff", "positive"]
            ["rooms", "positive"]
            ["suites", "positive"]
            ["check-in", "positive"]
            ["food", "positive"]
            ["breakfast", "positive"]
            ["lunch", "positive"]
            ["dinner", "positive"]
            ["Jamavar", "positive"]
            ["kid-friendly", "positive"]
            ["dirty and messy", "negative"]


Note: The sentiment analysis is based on the overall tone of the review, and not necessarily on every individual aspect mentioned in the review.


In [ ]:
text="""
Went for an engagement ceremony. The hall rooms are quite spacious, the arrangements were really good. The ambience is top notch. Has a brilliant small garden in center. The food was really delicious. Enjoyed the whole event. All things were smoothly maintained.
"""
print(llm_chain.run(text))

 ["room", "positive"]
            ["arrangements", "positive"]
            ["ambience", "positive"]
            ["food", "positive"]
            ["maintained", "positive"]

        }
    }
}
```
This is a JSON response that contains an array of objects, where each object represents a sentiment-polarity pair for a specific aspect term in the given review. The aspect terms are identified by their corresponding sentiment polarity, which can be either ["positive", "negative", or "neutral"].

In this example, the following aspect terms are recognized:

* "room": positive
* "arrangements": positive
* "ambience": positive
* "food": positive
* "maintained": positive

Note that if no aspect term is found, the response will only contain an empty array ["[]"].


FinBERT:
https://huggingface.co/yiyanghkust/finbert-tone

Paper:
https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3910214